In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

model_name = "/kaggle/input/final-urdu-model/pytorch/default/1"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

config.json:   0%|          | 0.00/989 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

In [21]:
from datasets import load_dataset

test_ds = load_dataset(
    "json",
    data_files={"test": "/kaggle/input/test-count/test.jsonl"}, 
    split="test"
)

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
def make_eval_prompt(question_ur):
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

آپ ایک پاکستانی آئینی قانون کے ماہر ہیں۔ دیے گئے سیاق و سباق کی بنیاد پر درست اور جامع جواب دیں۔<|eot_id|><|start_header_id|>user<|end_header_id|>

{question_ur}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    return prompt

In [16]:
def generate_answer(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    full_text = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # extract after last assistant header if present
    if "<|start_header_id|>assistant<|end_header_id|>" in full_text:
        ans_part = full_text.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
    else:
        ans_part = full_text

    answer_clean = ans_part.split("<|end_of_text|>")[0]
    answer_clean = answer_clean.replace("・━・━", "").replace("erusform", "").strip()
    
    return answer_clean


In [17]:
num_examples = 5  
subset = test_ds.select(range(num_examples))

for i, ex in enumerate(subset):
    prompt = make_eval_prompt(ex["instruction"])
    pred = generate_answer(prompt)

    print("="*60)
    print(f"Example {i+1}")
    print("Question (سوال/پرومپٹ):\n", ex["instruction"])
    print("\nActual Answer (اصل جواب):\n", ex["output"])
    print("\nModel Answer (ماڈل کا جواب):\n", pred)
    print()

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Example 1
Question (سوال/پرومپٹ):
 آپ ایک پاکستانی آئینی قانون کے ماہر ہیں۔ دیے گئے آئینی سیاق و سباق کی بنیاد پر سوال کا جواب دیں۔

سیاق و سباق:
کابینہ 1. گورنر کو اس کے کارہائے منصبی کی انجام دہی میں مدد اور مشورہ دینے کے لیے وزراء کی ایک کابینہ ہوگی، جس کا سربراہ وزیر اعلیٰ ہوگا۔ 2. صوبائی اسمبلی کا اجلاس اسمبلی کے عام انتخابات کے اکیس دن بعد ہوگا، تاآنکہ اس سے پہلے گورنر اجلاس طلب نہ کرلے۔ 3. اسپیکر اور ڈپٹی اسپیکر کے الیکشن کے بعد صوبائی اسمبلی، کسی بھی دوسری کارروائی کو چھوڑ کر، کسی بحث کے بغیر، اپنے اراکین میں سے ایک کو وزیر اعلیٰ کے طور پر منتخب کرے گی۔ 4. وزیر اعلیٰ کو صوبائی اسمبلی کے کل اراکین کی تعداد کی اکثریتی رائے دہی کے ذریعے نامزد کیا جائے گا: - مگر شرط یہ ہے کہ، اگر کوئی رکن پہلی رائے شماری میں مذکورہ اکثریت حاصل نہ کر سکے تو، پہلی رائے شماری میں سب سے زیادہ ووٹ حاصل کرنے والے پہلے دو اراکین کے درمیان دوسری رائے شماری کا انعقاد کیا جائے گا اور وہ رکن جو موجودہ اراکین کی اکثریتی رائے دہی حاصل کر لیتا ہے اس کا منتخب وزیر اعلیٰ کے طور پر اعلان کیا جائے گا: - مزید شرط یہ 

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Example 2
Question (سوال/پرومپٹ):
 آپ ایک پاکستانی آئینی قانون کے ماہر ہیں۔ دیے گئے آئینی سیاق و سباق کی بنیاد پر سوال کا جواب دیں۔

سیاق و سباق:
مسلح افواج کی کمان [243. (1) وفاقی حکومت کے پاس مسلح افواج کی کمان اور کنٹرول ہوگا۔ قبل الذکر حکم کی عمومیت پر اثر انداز ہوئے بغیر، مسلح افواج کی اعلیٰ کمان صدر کے ہاتھ میں ہوگی۔ صدر کو قانون کے تابع، یہ اختیار ہوگا کہ وہ: الف) پاکستان کی بری، بحری اور فضائی افواج اور مذکورہ افواج کے محفوظ دستے قائم کرے اور ان کی دیکھ بھال کرے؛ اور ب) مذکورہ افواج میں کمیشن عطا کرے۔ صدر، وزیراعظم کے ساتھ مشورے پر: الف) چیئرمین جوائنٹ چیفس آف اسٹاف کمیٹی؛ ب) چیف آف آرمی اسٹاف؛ ج) چیف آف نیول اسٹاف؛ اور د) چیف آف ایئر اسٹاف، کا تقرر کرے گا اور ان کی تنخواہوں اور الاؤنسوں کا تعین بھی کرے گا۔]

سوال:
پاکستان میں مسلح افواج کی حتمی کمان اور کنٹرول کس کے پاس ہے؟

جواب:

Actual Answer (اصل جواب):
 پاکستان میں مسلح افواج کی کمان اور کنٹرول وفاقی حکومت کے پاس ہے۔

Model Answer (ماڈل کا جواب):
 وفاقی حکومت کے پاس مسلح افواج کی کمان اور کنٹرول ہے۔



The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Example 3
Question (سوال/پرومپٹ):
 آپ ایک پاکستانی آئینی قانون کے ماہر ہیں۔ دیے گئے آئینی سیاق و سباق کی بنیاد پر سوال کا جواب دیں۔

سیاق و سباق:
عدالت عظمیٰ کے فیصلوں کا جائزہ 1. نیا آرٹیکل 186A پی او نمبر 14 آف 1985، آرٹیکل 2 اور شیڈول کے ذریعے شامل کیا گیا۔ 2. آئین (پانچویں ترمیم) ایکٹ، 1976 (62 آف 1976)، سیکشن 7 کے ذریعے "دی" کے لیے تبدیل کیا گیا (13 ستمبر، 1976 سے مؤثر)۔ عدالت عظمیٰ کو کسی بھی ایکٹ 1[مجلس شوریٰ (پارلیمنٹ)] اور عدالت عظمیٰ کے وضع کردہ کسی قواعد کے احکام کے تابع، اپنے صادر کردہ کسی فیصلے یا دیے ہوئے کسی حکم پر نظرثانی کرنے کا اختیار ہوگا۔

سوال:
کیا پاکستان کی عدالت عظمیٰ اپنے فیصلوں کا جائزہ لے سکتی ہے؟

جواب:

Actual Answer (اصل جواب):
 جی ہاں، پاکستان کی عدالت عظمیٰ کو اپنے صادر کردہ کسی فیصلے یا دیے ہوئے کسی حکم پر نظرثانی کرنے کا اختیار ہے۔

Model Answer (ماڈل کا جواب):
 جی ہاں، عدالت عظمیٰ کو ایکٹ 1[مجلس شوریٰ (پارلیمنٹ)] اور عدالت عظمیٰ کے وضع کردہ کسی قواعد کے احکام کے تابع اپنے صادر کردہ کسی فیصلے یا دیے ہوئے کسی حکم پر نظرثانی کرنے کا اختیار ہے۔asser



The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Example 4
Question (سوال/پرومپٹ):
 آپ ایک پاکستانی آئینی قانون کے ماہر ہیں۔ دیے گئے آئینی سیاق و سباق کی بنیاد پر سوال کا جواب دیں۔

سیاق و سباق:
صدر، وغیرہ کی تنخواہیں، بھتے، وغیرہ (1) یوم آغاز سے دو سال کے اندر اندر، صدر، اسپیکر اور ڈپٹی اسپیکر اور قومی اسمبلی یا کسی صوبائی اسمبلی کے رکن، چیئرمین اور ڈپٹی چیئرمین اور سینیٹ کے رکن، وزیر اعظم، کسی وفاقی وزیر، وزیر مملکت، کسی وزیر اعلیٰ، کسی صوبائی وزیر اور چیف الیکشن کمشنر کی تنخواہیں، بھتے اور مراعات کے تعین کے لیے قانون کے ذریعے احکام وضع کیے جائیں گے۔ جب تک کہ قانون کے ذریعے دیگر احکام وضع نہ کیے جائیں،— الف) صدر، قومی اسمبلی یا کسی صوبائی اسمبلی کے اسپیکر یا ڈپٹی اسپیکر یا رکن، کسی وفاقی وزیر، وزیر مملکت، کسی وزیر اعلیٰ، کسی صوبائی وزیر اور چیف الیکشن کمشنر کی تنخواہیں، بھتے اور مراعات وہی ہوں گی جن کا صدر، قومی اسمبلی پاکستان یا کسی صوبائی اسمبلی کا اسپیکر، ڈپٹی اسپیکر یا رکن، کوئی وفاقی وزیر، کوئی وزیر مملکت، کوئی وزیر اعلیٰ، کوئی صوبائی وزیر، یا جیسی بھی صورت ہو، چیف الیکشن کمشنر یوم آغاز سے عین قبل مستحق تھا؛ اور ب) چیئرمین، ڈپ

In [18]:
preds = []
refs = []

for ex in test_ds:
    prompt = make_eval_prompt(ex["instruction"])
    pred = generate_answer(prompt)
    preds.append(pred)
    refs.append(ex["output"])

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

In [19]:
import numpy as np

def f1_score(pred, ref):
    pred_tokens = pred.split()
    ref_tokens = ref.split()
    common = set(pred_tokens) & set(ref_tokens)
    if not common:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(ref_tokens)
    return 2 * precision * recall / (precision + recall)

scores = [f1_score(p, r) for p, r in zip(preds, refs)]
print("Mean F1 over test set:", np.mean(scores))

Mean F1 over test set: 0.6175362222226545


In [20]:
!pip install nltk
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [32]:
from nltk.translate.meteor_score import meteor_score
import numpy as np

In [23]:
def meteor_ur(pred, ref):
    pred_tokens = pred.split()
    ref_tokens = ref.split()
    return meteor_score([ref_tokens], pred_tokens)

meteor_scores = [meteor_ur(p, r) for p, r in zip(preds, refs)]
print("Mean METEOR:", np.mean(meteor_scores))

Mean METEOR: 0.6745206915264405


In [31]:
!pip install sacrebleu
import sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.1 MB/s eta 0:00:00


In [26]:
bleu = sacrebleu.corpus_bleu(preds, [refs])
print("SacreBLEU:", bleu.score)

SacreBLEU: 50.0350408083491


# RAG Based Implementation 

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 70.4 MB/s eta 0:00:00:00:0100:01


In [3]:
import pandas as pd
import numpy as np
import faiss
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

print("Loading knowledge base, embeddings, index, and model...")

# 1) Knowledge base
unique_contexts = pd.read_csv("/kaggle/input/unique-contexts/unique_contexts.csv")
contexts_list = unique_contexts["Context_urdu_clean"].tolist()
context_ids = np.load("/kaggle/input/models/pytorch/default/1/context_ids.npy")

# 2) FAISS index
index = faiss.read_index("/kaggle/input/models/pytorch/default/1/constitutional_law_index.faiss")

# 3) Generation model (base + LoRA adapter)
model_name = "/kaggle/input/final-urdu-model/pytorch/default/1"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

llm_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

llm_model.eval()
print("✓ Initialization complete.")

Loading knowledge base, embeddings, index, and model...


config.json:   0%|          | 0.00/989 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

✓ Initialization complete.


## Retriever Helper

In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1) Fit TF‑IDF on all contexts
vectorizer = TfidfVectorizer()
context_vecs = vectorizer.fit_transform(contexts_list)

def retrieve_contexts(question_ur, top_k=3):
    """Return top‑k contexts for a question using TF‑IDF cosine similarity."""
    q_vec = vectorizer.transform([question_ur])
    sims = cosine_similarity(q_vec, context_vecs)[0]
    top_idx = sims.argsort()[::-1][:top_k]
    results = []
    for rank, idx in enumerate(top_idx, start=1):
        results.append({
            "rank": rank,
            "context_index": idx,
            "context_text": contexts_list[idx],
            "similarity_score": float(sims[idx]),
        })
    return results

## Prompt Building + Generation

In [18]:
import re

def clean_answer(raw_text: str) -> str:
    # 1) keep only assistant part
    if "<|start_header_id|>assistant<|end_header_id|>" in raw_text:
        ans = raw_text.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
    else:
        ans = raw_text

    # 2) cut at end_of_text
    ans = ans.split("<|end_of_text|>")[0]

    # 3) remove ANY reserved_special_token_<number>
    ans = re.sub(r"<\|reserved_special_token_\d+\|>", "", ans)

    # 4) drop other junk and strip
    for junk in ["erusform", "・━・━"]:
        ans = ans.replace(junk, "")
    return ans.strip()

In [19]:
def build_rag_prompt(question_ur, context_ur):
    """Match your fine‑tuning format: system + context + question + جواب."""
    instruction = f"""آپ ایک پاکستانی آئینی قانون کے ماہر ہیں۔ دیے گئے آئینی سیاق و سباق کی بنیاد پر سوال کا جواب دیں۔

سیاق و سباق:
{context_ur}

سوال:
{question_ur}

جواب:"""

    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

آپ ایک پاکستانی آئینی قانون کے ماہر ہیں۔ دیے گئے سیاق و سباق کی بنیاد پر درست اور جامع جواب دیں۔<|eot_id|><|start_header_id|>user<|end_header_id|>

{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    return prompt


def generate_answer_from_prompt(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(llm_model.device)

    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.3,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=False)

    return clean_answer(text)

## RAG Query Function

In [13]:
def rag_query(question_ur, top_k=3, verbose=True):
    if verbose:
        print("=" * 60)
        print("سوال:", question_ur)
        print("=" * 60)

    # 1) Retrieve
    contexts = retrieve_contexts(question_ur, top_k=top_k)

    if verbose:
        print("\nRetrieved contexts:")
        for c in contexts:
            print(f"\nRank {c['rank']} | ID {c['context_index']} | score {c['similarity_score']:.4f}")
            print("Text:", c["context_text"][:120], "...")

    # 2) Use top‑1 context for now
    top_context = contexts[0]["context_text"]

    # 3) Build prompt + generate
    prompt = build_rag_prompt(question_ur, top_context)
    answer = generate_answer_from_prompt(prompt)

    if verbose:
        print("\nGenerated answer:\n", answer)
        print("\n" + "=" * 60)

    return answer, contexts

In [20]:
answer, ctxs = rag_query("اگر پہلی رائے شماری میں کوئی رکن اکثریت حاصل نہ کرے تو کیا ہوتا ہے؟", top_k=3)

سوال: اگر پہلی رائے شماری میں کوئی رکن اکثریت حاصل نہ کرے تو کیا ہوتا ہے؟

Retrieved contexts:

Rank 1 | ID 1 | score 0.3392
Text: صدر کو اس کے کارہائے منصبی کی انجام دہی میں مدد اور مشورہ دینے کے لیے وزراء کی ایک کابینہ ہوگی جس کا سربراہ وزیراعظم ہوگ ...

Rank 2 | ID 160 | score 0.2900
Text: کابینہ 1. گورنر کو اس کے کارہائے منصبی کی انجام دہی میں مدد اور مشورہ دینے کے لیے وزراء کی ایک کابینہ ہوگی، جس کا سربراہ ...

Rank 3 | ID 166 | score 0.2048
Text: حساب پر رائے شماری مذکورہ بالا احکام میں مالی امور سے متعلق شامل کسی امر کے باوجود، صوبائی اسمبلی کو اختیار ہوگا کہ وہ ت ...

Generated answer:
 اگر پہلی رائے شماری میں کوئی رکن اکثریت حاصل نہ کرے تو پہلے دو اراکین کے درمیان میں دوسری رائے شماری کا انعقاد کیا جاتا ہے اور وہ رکن جو موجودہ اراکین کی اکثریت رائے دہی حاصل کر لیتا ہے اس کا منتخب وزیراعظم کے طور پر اعلان کیا جاتا ہے۔



In [24]:
def extract_gold_context(instr: str) -> str:
    # between "سیاق و سباق:" and "سوال:"
    if "سیاق و سباق:" in instr and "سوال:" in instr:
        part = instr.split("سیاق و سباق:")[1]
        ctx = part.split("سوال:")[0]
        return ctx.strip()
    return ""

In [27]:
def hit_at_k(test_ds, k=3):
    hits = 0
    total = 0

    for ex in test_ds:
        instr = ex["instruction"]
        gold_ctx = extract_gold_context(instr)
        if not gold_ctx:
            continue

        # get question text from instruction
        q = instr.split("سوال:")[-1].split("جواب:")[0].strip()

        retrieved = retrieve_contexts(q, top_k=k)
        retrieved_texts = [r["context_text"].strip() for r in retrieved]

        if gold_ctx in retrieved_texts:
            hits += 1
        total += 1

    return hits / total if total > 0 else 0.0

In [28]:
hit1 = hit_at_k(test_ds, k=1)
hit3 = hit_at_k(test_ds, k=3)
hit5 = hit_at_k(test_ds, k=5)

print("Hit@1:", hit1)
print("Hit@3:", hit3)
print("Hit@5:", hit5)

Hit@1: 0.6914893617021277
Hit@3: 0.8936170212765957
Hit@5: 0.9468085106382979


In [29]:
import re

def extract_question(instr: str) -> str:
    if "سوال:" in instr:
        return instr.split("سوال:")[-1].split("جواب:")[0].strip()
    return instr.strip()

preds_rag = []
refs = []

for ex in test_ds:
    instr = ex["instruction"]
    gold = ex["output"]

    q = extract_question(instr)
    pred, _ = rag_query(q, top_k=3, verbose=False)

    preds_rag.append(pred)
    refs.append(gold)

In [33]:
import numpy as np
from nltk.translate.meteor_score import meteor_score
import sacrebleu

def meteor_ur(pred, ref):
    return meteor_score([ref.split()], pred.split())

meteor_scores = [meteor_ur(p, r) for p, r in zip(preds_rag, refs)]
print("Mean METEOR (RAG):", np.mean(meteor_scores))

bleu = sacrebleu.corpus_bleu(preds_rag, [refs])
print("SacreBLEU (RAG):", bleu.score)

Mean METEOR (RAG): 0.582817911710239
SacreBLEU (RAG): 41.08589353248347
